### Part 5 Building a Representative Snowpack at the 100 m scale
[Benoit Montpetit](https://github.com/ecccben), *CPS/CRD/ECCC*, 2024  
[Mike Brady](https://github.com/m9brady), *CPS/CRD/ECCC*, 2024


To characterize the snowpack at the 100 m scale the SMP profile (N~18) having the snow depth closest to the median snow depth of the MagnaProbe distribution (N~290) was selected to simulate the radar backscatter at that scale.

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import gamma
import pickle

from matplotlib import pyplot as plt
import matplotlib

font = {'family' : 'sans-serif',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)
plt.rcParams["axes.labelsize"] = 22
plt.rcParams["axes.labelweight"] = 'bold'
plt.rcParams['xtick.labelsize']=16
plt.rcParams['ytick.labelsize']=16

pd.set_option('use_inf_as_na', True)

In [ ]:
smp_profiles = pd.read_pickle(r"data/smp_result.pkl")
ref_ssa = pd.read_pickle(r"data/ref_ssa.pkl")
ref_temp = pd.read_pickle(r"data/ref_temp.pkl")
mp = pd.read_pickle(r'data/ref_mp.pkl')

Some discrepenties between the sites naming conventions were noted between the work of [Josh King](https://github.com/kingjml) and [Benoit Montpetit](https://github.com/ecccben). The site names were not changed in the files but the naming convention used here is the one found in [Montpetit et al. (2024)](Link TBD)


In [ ]:
sites = smp_profiles.site.unique()
sites=[s.replace('RS','RP') for s in sites]
sites.remove('SR01')
sites.remove('SC01')
sites.remove('SC02')
sites.remove('SC03')

In [ ]:

svclassifier = pickle.load(open(r'data/SVClassifierTVC02_2.pkl', 'rb'))

# Building the 5 cm aggregated SMRT ready pit dataframes with the classified grain types

In [ ]:
pits = {}

for site in sites:
    
    pits[site] = {}

    if 'RP' in site:
    
        profiles = smp_profiles.loc[smp_profiles.site==site.replace('RP','RS')]
        temps = ref_temp.loc[ref_temp.site==site.replace('RP','RS')]

    else:

        profiles = smp_profiles.loc[smp_profiles.site==site]
        temps = ref_temp.loc[ref_temp.site==site]

    for position in profiles.position:

        if position in ['SSA','RHO','S1','S2','S3','S4','S5',
                        'N1','N2','N3','N4','N5',
                        'W1','W2','W3','E1','E2','E3']:

            profile = profiles.loc[profiles.position==position].dropna()
            profile.iloc[0].density['temp']=profile.iloc[0].density.apply(lambda x: temps.loc[(x.rel_height - temps.rel_height).abs().idxmin()].temperature_k, axis=1)

            thickness = profile.iloc[0].density.dropna().rel_height.groupby(
                np.arange(len(profile.iloc[0].density.dropna().density))//21).apply(np.ptp).values

            if thickness[0]>=50:

                pits[site][position]={}



                if thickness[-1]<40:

                    idx = profile.iloc[0].density.dropna().rel_height.groupby(
                            np.arange(len(profile.iloc[0].density.dropna().density))//21).groups
                    idx[list(idx.keys())[-2]] = np.concatenate((idx[list(idx.keys())[-2]],idx[list(idx.keys())[-1]]))
                    del idx[list(idx.keys())[-1]]

                    thickness = np.array([])
                    density = np.array([])
                    ssa = np.array([])
                    force_mean = np.array([])
                    force_std = np.array([])
                    rel_height = np.array([])
                    temperature = np.array([])

                    for item in idx:

                        thickness = np.append(thickness,np.ptp(profile.iloc[0].density.dropna().rel_height[list(idx[item])].values))
                        density = np.append(density, np.median(profile.iloc[0].density.dropna().density[list(idx[item])].values))
                        ssa = np.append(ssa, np.median(profile.iloc[0].ssa.dropna().ssa[list(idx[item])].values))
                        force_mean = np.append(force_mean, np.median(profile.iloc[0].density.dropna().force_median[list(idx[item])].values))
                        force_std = np.append(force_std, np.median(profile.iloc[0].density.dropna().l[list(idx[item])].values))
                        rel_height = np.append(rel_height, profile.iloc[0].density.dropna().rel_height[list(idx[item])].values.max())
                        temperature = np.append(temperature, np.median(profile.iloc[0].density.dropna().temp[list(idx[item])].values))

                else:

                    density = profile.iloc[0].density.dropna().density.groupby(np.arange(len(profile.iloc[0].density.dropna().density))//21).median().values
                    ssa = profile.iloc[0].ssa.dropna().ssa.groupby(np.arange(len(profile.iloc[0].density.dropna().density))//21).median().values
                    force_mean = profile.iloc[0].density.dropna().force_median.groupby(
                                        np.arange(len(profile.iloc[0].density.dropna().density))//21).median().values
                    force_std = profile.iloc[0].density.dropna().l.groupby(
                                        np.arange(len(profile.iloc[0].density.dropna().density))//21).median().values
                    rel_height = profile.iloc[0].density.dropna().rel_height.groupby(
                                        np.arange(len(profile.iloc[0].density.dropna().density))//21).max().values
                    temperature = profile.iloc[0].density.dropna().temp.groupby(
                                        np.arange(len(profile.iloc[0].density.dropna().density))//21).median().values
                    

                pits[site][position]['thickness']=thickness/1000
                pits[site][position]['density']=density
                pits[site][position]['ssa']=ssa
                pits[site][position]['temperature']=temperature
                pits[site][position]['p_ex']=4/917/ssa*(1-density/917)
                pits[site][position]['force_median']=force_mean
                pits[site][position]['force_std']=force_std
                pits[site][position]['rel_height']=rel_height
                pits[site][position]['grain_type']=svclassifier.predict(pd.DataFrame({'rel_height':rel_height, 
                                                        'F_med':force_mean, 'L':force_std}))

df_aggpits = pd.DataFrame(pits)

In [ ]:
df_aggpits.to_pickle(r'data/df_aggpits.pkl')

# Selecting the SMP profile with the total snow depth closest to the median value of the distributed MagnaProbe snow depths

In [ ]:
smp_profiles_mode = pd.DataFrame()
pits_mode = {}

x = np.linspace(0,100,200)

for site in sites:
    
    mp_site = mp.loc[(mp['site']==site.replace('RP','RS'))].depth
    param = gamma.fit(mp_site)
    sd_mode = gamma.pdf(x, *param[:-2], loc = param[-2], scale = param[-1]).argmax()
    
    i=0
    best_pit = smp_profiles[(smp_profiles.site==site.replace('RP','RS'))].position.iloc[i]
    delta_mode = np.abs(smp_profiles[(smp_profiles.site==site.replace('RP','RS'))].iloc[i].density.rel_height.max()/10-sd_mode)
    
    for i in range(len(smp_profiles[(smp_profiles.site==site.replace('RP','RS'))])):
        
        if ((np.abs(smp_profiles[(smp_profiles.site==site.replace('RP','RS'))].iloc[i].density.rel_height.max()/10-sd_mode) < delta_mode) and
        (smp_profiles[(smp_profiles.site==site.replace('RP','RS'))].position.iloc[i] in ['SSA','RHO','S1','S2','S3','S4','S5',
                                                                                        'N1','N2','N3','N4','N5',
                                                                                        'W1','W2','W3','E1','E2','E3'])):
            
            best_pit = smp_profiles[(smp_profiles.site==site.replace('RP','RS'))].position.iloc[i]
            delta_mode = np.abs(smp_profiles[(smp_profiles.site==site.replace('RP','RS'))].iloc[i].density.rel_height.max()/10-sd_mode)
        
    
    smp_profiles_mode = pd.concat([smp_profiles_mode, smp_profiles.loc[(smp_profiles.site==site.replace('RP','RS')) & 
                                                                 (smp_profiles.position==best_pit)]])
    pits_mode[site] = df_aggpits[site][best_pit]
    
df_pits_mode = pd.DataFrame(pits_mode)

In [ ]:
df_pits_mode.to_pickle(r'data/df_pits_mode.pkl')

In [ ]:

fig, ax = plt.subplots(int(np.ceil(len(sites)/3)),3,figsize=(15, 100))
x = np.linspace(0,100,200)
i=0
j=0

for site in sites:

    mp_site = mp.loc[(mp.site==site.replace('RP','RS'))].depth
    param = gamma.fit(mp_site)
    sd_mode = x[gamma.pdf(x, *param[:-2], loc = param[-2], scale = param[-1]) == gamma.pdf(x, *param[:-2], loc = param[-2], scale = param[-1]).max()]
        
    best_pit = smp_profiles[(smp_profiles.site==site.replace('RP','RS'))].iloc[0].position
    delta_mode = np.abs(smp_profiles[(smp_profiles.site==site.replace('RP','RS'))].iloc[0].density.rel_height.max()/10-sd_mode)

    for position in smp_profiles[(smp_profiles.site==site.replace('RP','RS'))].position.values:
        
        if np.abs(smp_profiles[(smp_profiles.site==site.replace('RP','RS')) & (smp_profiles.position==position)].density.iloc[0].rel_height.max()/10-sd_mode) < delta_mode:
            
            best_pit = position
            delta_mode = np.abs(smp_profiles[(smp_profiles.site==site.replace('RP','RS')) & (smp_profiles.position==position)].density.iloc[0].rel_height.max()/10-sd_mode)

    hsnow = ax[i,j].hist(mp_site, bins=25, density=True, label='MagnaProbe Depths')
    ax[i,j].plot(x,gamma.pdf(x, *param[:-2], loc = param[-2], scale = param[-1]), label='Fitted Gamma Distribution',
            linewidth=5)

    for position in smp_profiles.loc[(smp_profiles.site==site.replace('RP','RS'))].position.values:
        ax[i,j].axvline(smp_profiles.loc[(smp_profiles.site==site.replace('RP','RS')) & (smp_profiles.position==position) ].density.iloc[0].rel_height.max()/10,color='gray',linestyle='--',label='SMP snow depth')
    # ax[i,j].axvline(smp_profiles.loc[(smp_profiles.site==site.replace('RP','RS')) & (smp_profiles.position==position) ].density.iloc[0].rel_height.max()/10,0.04,facecolor='gray')
    ax[i,j].axvline(smp_profiles.loc[(smp_profiles.site==site.replace('RP','RS')) & (smp_profiles.position==best_pit) ].density.iloc[0].rel_height.max()/10,color='r',label='Best SMP snow depth')
    if i==int(np.ceil(len(sites)/3)):
        ax[i,j].set_xlabel('Snow Depth (cm)', fontsize=16, fontweight='bold')
    if j==0 and i==13:
        ax[i,j].set_ylabel('Probability Density Function', fontsize=16, fontweight='bold')
    ax[i,j].set_title(site, fontsize=16, fontweight='bold')
    ax[i,j].set_ylim(0,0.06)

    #a = plt.gca()
    ax[i,j].set_xticklabels(ax[i,j].get_xticks(), fontsize=12,fontweight='bold')
    ax[i,j].set_yticklabels(ax[i,j].get_yticks(), fontsize=12,fontweight='bold')

    if j==2:
        i+=1
        j=0
    else:
        j+=1

handles, labels = ax[0,0].get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax[0,0].legend(by_label.values(), by_label.keys(), prop={'size':16,'weight':'bold'},bbox_to_anchor=(0.55,1.45),ncols=2)


<center><img src="Figures/Part_5_RepPit_Fig1.png" Width="1000px"></center>

<center>Figure: Selected SMP profile snow depth (red) compared to all SMP profiles depths (gray) and the distributed Magna Probe snow depths (blue). Orange line is the Gamma distribution fitted on the MagnaProbe data</center>

## **************************************************************************

# Analysis of Snowpack geophysical properties for all three intensive campaign of the TVC Experiment 2018/19

In [ ]:
from constants import TVC01,TVC02,TVC03
sites = pd.DataFrame({'site':[TVC01+TVC02+TVC03]})
sites.site.replace({'RS':'RP'}, regex=True, inplace=True)
sites=list(sites.site.values)

In [ ]:
depth_stats = pd.DataFrame()

for camp in [TVC01,TVC02,TVC03]:

    depth_stats = pd.concat([depth_stats,pd.DataFrame({('Depth','Median'):[mp.loc[mp.site.isin(camp)].depth.median()], ('Depth','std'): [mp.loc[mp.site.isin(camp)].depth.std()]})],ignore_index=True)

depth_stats=depth_stats.rename(index={0:'Nov.',1:'Jan.',2:'Mar.'})
depth_stats

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead tr th {
        text-align: left;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr>
      <th></th>
      <th colspan="2" halign="left">Depth</th>
    </tr>
    <tr>
      <th></th>
      <th>Median</th>
      <th>std</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Nov.</th>
      <td>33.15</td>
      <td>9.535905</td>
    </tr>
    <tr>
      <th>Jan.</th>
      <td>45.61</td>
      <td>13.213402</td>
    </tr>
    <tr>
      <th>Mar.</th>
      <td>42.48</td>
      <td>14.033336</td>
    </tr>
  </tbody>
</table>
</div>

In [ ]:
camps = ['TVC01','TVC02','TVC03']
types = ['R','H']

h_frac = []
h = []
ssa = {}
ssa['R'] = []
ssa['H'] = []
rho = {}
rho['R'] = []
rho['H'] = []
temp = {}
temp['R'] = []
temp['H'] = []
campaign = []
campaign2 = {}
campaign2['R'] = []
campaign2['H'] = []

i=0


for camp in [TVC01,TVC02,TVC03]:

    for site in camp:
    
        if site.replace('RS', 'RP') in df_aggpits.keys():
        
            pits = df_aggpits.loc[:,site.replace('RS', 'RP')].dropna()

            for pit in pits.keys():

                profile = pits[pit]

                if (profile['grain_type']=='H').any()==True:

                    h_frac.extend([profile['thickness'][profile['grain_type']=='H'].sum()/profile['thickness'].sum()])
                    h.extend([profile['thickness'].sum()])

                    for gtype in types:
                        ssa[gtype].extend(profile['ssa'][profile['grain_type']==gtype])
                        rho[gtype].extend(profile['density'][profile['grain_type']==gtype])
                        temp[gtype].extend(profile['temperature'][profile['grain_type']==gtype])
                        campaign2[gtype].extend([i]*len(profile['temperature'][profile['grain_type']==gtype]))

                else:

                    gtype='R'

                    h_frac.extend([0.])
                    h.extend([profile['thickness'].sum()])
                    ssa[gtype].extend(profile['ssa'][profile['grain_type']==gtype])
                    rho[gtype].extend(profile['density'][profile['grain_type']==gtype])
                    temp[gtype].extend(profile['temperature'][profile['grain_type']==gtype])
                    campaign2[gtype].extend([i]*len(profile['temperature'][profile['grain_type']==gtype]))

                campaign.extend([i])

    i+=1

h_frac=np.array(h_frac)
h=np.array(h)
campaign=np.array(campaign)
for gtype in types:
    ssa[gtype]=np.array(ssa[gtype])
    rho[gtype]=np.array(rho[gtype])
    temp[gtype]=np.array(temp[gtype])
    campaign2[gtype]=np.array(campaign2[gtype])


In [ ]:
fig, ax = plt.subplots(2,3,figsize=(25,10))

cols=['k','gray','cyan']
camps = ['Nov.','Jan.','Mar.']

for i in range(3):

    

    if i==0:
        j=0
        for gtype in types:
            ssasnow = ax[j,0].hist(ssa[gtype][(campaign2[gtype]==i) & (ssa[gtype]<=60)], bins=range(0,60,2), density=True, label=camps[i], facecolor=cols[i],edgecolor='k')
            rhosnow = ax[j,1].hist(rho[gtype][(campaign2[gtype]==i)], bins=range(50,450,10), density=True, label=camps[i], facecolor=cols[i],edgecolor='k')
            tsnow = ax[j,2].hist(temp[gtype][(campaign2[gtype]==i)], bins=range(235,275), density=True, label=camps[i], facecolor=cols[i],edgecolor='k')
            j+=1   

            
    else:
        j=0
        for gtype in types:
            ssasnow = ax[j,0].hist(ssa[gtype][(campaign2[gtype]==i) & (ssa[gtype]<=60)], bins=range(0,60,2), density=True, label=camps[i], facecolor=cols[i], edgecolor='k',alpha=0.5)
            ssasnow = ax[j,1].hist(rho[gtype][(campaign2[gtype]==i)], bins=range(50,450,10), density=True, label=camps[i], facecolor=cols[i], edgecolor='k',alpha=0.5)
            ssasnow = ax[j,2].hist(temp[gtype][(campaign2[gtype]==i)], bins=range(235,275), density=True, label=camps[i], facecolor=cols[i], edgecolor='k',alpha=0.5)
            j+=1

    for j in range(2):
        ax[j,0].set_xlim(0,60)
        ax[j,1].set_xlim(50,450)
        ax[j,2].set_xlim(235,275)
ax[0,0].set_ylabel('PDF \nrounded grains',fontsize=20, fontweight='bold')
ax[1,0].set_ylabel('PDF \ndepth hoar',fontsize=20, fontweight='bold')
ax[0,0].legend()
ax[j,0].set_xlabel('SSA ($m^2\\cdot kg^{-1}$)',fontsize=20, fontweight='bold')
ax[j,1].set_xlabel('$\\rho_{snow}$ ($kg\\cdot m^{-3}$)',fontsize=20, fontweight='bold')
ax[j,2].set_xlabel('$T_{snow}$ ($^o$C)',fontsize=20, fontweight='bold')



<center><img src="Figures/Figure5.png" height="500px" class="bg-primary mb-1"></center>

<center>Figure 5 of [Montpetit et al. (2024)](Link TBD): Evolution of snow geophysical properties (SSA, density, and temperature) throughout the winter season for the two dominant snow
grain type layers: rounded (R) and depth hoar grains (H).</center>

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,10))

camps = ['Nov.','Jan.','Mar.']
cols=['k','gray','cyan']

i=0

for camp in [TVC01,TVC02,TVC03]:

    mp_site = mp.loc[mp.site.isin(camp)].depth
    if i==0:
        hsnow = ax[0].hist(mp_site, bins=range(0,125,5), density=True, label=camps[i], color=cols[i], edgecolor='k')
        hfracsnow = ax[1].hist(h_frac[campaign==i], bins=np.arange(0,1.01,0.05), density=True, label=camps[i], color=cols[i], edgecolor='k')
    else:
        hsnow = ax[0].hist(mp_site, bins=range(0,125,5), density=True, label=camps[i], color=cols[i],alpha=0.5, edgecolor='k')
        hfracsnow = ax[1].hist(h_frac[campaign==i], bins=np.arange(0,1.01,0.05), density=True, label=camps[i], color=cols[i],alpha=0.5, edgecolor='k')

    i+=1

ax[0].legend()
ax[0].set_xlim(0,120)
ax[1].set_xlim(0,1)
ax[0].set_xlabel('Snow depth (cm)',fontsize=20, fontweight='bold')
ax[1].set_xlabel('Depth hoar fraction',fontsize=20, fontweight='bold')
ax[0].set_ylabel('Probability density function',fontsize=20, fontweight='bold')


<center><img src="Figures/Figure4.png" height="500px" class="bg-primary mb-1"></center>

<center>Figure 4 of [Montpetit et al. (2024)](Link TBD): Evolution of snow depth distributions and depth hoar fraction throughout the field campaign.</center>

# Compiling the SSA, density and temperature for the different snow layers per measurement campaign

In [ ]:
col_names = pd.MultiIndex.from_tuples([('R','SSA','Median'),('R','SSA','std'),('R','Density','Median'),('R','Density','std'),('R','Temp','Median'),('R','Temp','std'),
                                       ('H','SSA','Median'),('H','SSA','std'),('H','Density','Median'),('H','Density','std'),('H','Temp','Median'),('H','Temp','std')])

camp_stats = pd.DataFrame(np.random.randn(3,12), columns=col_names, index=camps)



for i in range(3):
    for gtype in types:
        camp_stats.loc[camps[i],(gtype[0],'SSA','Median')]=np.median(ssa[gtype][(campaign2[gtype]==i) & (ssa[gtype]<=60)])
        camp_stats.loc[camps[i],(gtype[0],'SSA','std')]=ssa[gtype][(campaign2[gtype]==i) & (ssa[gtype]<=60)].std()
        camp_stats.loc[camps[i],(gtype[0],'Density','Median')]=np.median(rho[gtype][(campaign2[gtype]==i)])
        camp_stats.loc[camps[i],(gtype[0],'Density','std')]=rho[gtype][(campaign2[gtype]==i)].std()
        camp_stats.loc[camps[i],(gtype[0],'Temp','Median')]=np.median(temp[gtype][(campaign2[gtype]==i)])
        camp_stats.loc[camps[i],(gtype[0],'Temp','std')]=temp[gtype][(campaign2[gtype]==i)].std()

camp_stats

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead tr th {
        text-align: left;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr>
      <th></th>
      <th colspan="6" halign="left">R</th>
      <th colspan="6" halign="left">H</th>
    </tr>
    <tr>
      <th></th>
      <th colspan="2" halign="left">SSA</th>
      <th colspan="2" halign="left">Density</th>
      <th colspan="2" halign="left">Temp</th>
      <th colspan="2" halign="left">SSA</th>
      <th colspan="2" halign="left">Density</th>
      <th colspan="2" halign="left">Temp</th>
    </tr>
    <tr>
      <th></th>
      <th>Median</th>
      <th>std</th>
      <th>Median</th>
      <th>std</th>
      <th>Median</th>
      <th>std</th>
      <th>Median</th>
      <th>std</th>
      <th>Median</th>
      <th>std</th>
      <th>Median</th>
      <th>std</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Nov.</th>
      <td>21.947939</td>
      <td>6.986666</td>
      <td>193.7</td>
      <td>35.075993</td>
      <td>256.250000</td>
      <td>3.375242</td>
      <td>14.453362</td>
      <td>4.088281</td>
      <td>212.3</td>
      <td>21.371769</td>
      <td>261.049988</td>
      <td>3.878828</td>
    </tr>
    <tr>
      <th>Jan.</th>
      <td>31.698937</td>
      <td>8.672425</td>
      <td>334.0</td>
      <td>80.083839</td>
      <td>245.399994</td>
      <td>4.203806</td>
      <td>12.413849</td>
      <td>3.513907</td>
      <td>229.2</td>
      <td>20.242312</td>
      <td>253.250000</td>
      <td>5.052915</td>
    </tr>
    <tr>
      <th>Mar.</th>
      <td>20.703624</td>
      <td>4.750718</td>
      <td>320.5</td>
      <td>56.122843</td>
      <td>264.350006</td>
      <td>2.567876</td>
      <td>10.972934</td>
      <td>3.958132</td>
      <td>233.2</td>
      <td>23.001256</td>
      <td>264.949982</td>
      <td>2.008354</td>
    </tr>
  </tbody>
</table>
</div>

# Compiling the snow depth and depth hoar fraction for the different measurement campaigns

In [ ]:
for i in range(3):

    depth_stats.loc[camps[i],('h_frac','Median')]=np.median(h_frac[campaign==i])
    depth_stats.loc[camps[i],('h_frac','std')]=np.std(h_frac[campaign==i])

depth_stats

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead tr th {
        text-align: left;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr>
      <th></th>
      <th colspan="2" halign="left">Depth</th>
      <th colspan="2" halign="left">h_frac</th>
    </tr>
    <tr>
      <th></th>
      <th>Median</th>
      <th>std</th>
      <th>Median</th>
      <th>std</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Nov.</th>
      <td>33.15</td>
      <td>9.535905</td>
      <td>0.558824</td>
      <td>0.138767</td>
    </tr>
    <tr>
      <th>Jan.</th>
      <td>45.61</td>
      <td>13.213402</td>
      <td>0.639640</td>
      <td>0.140107</td>
    </tr>
    <tr>
      <th>Mar.</th>
      <td>42.48</td>
      <td>14.033336</td>
      <td>0.693093</td>
      <td>0.150799</td>
    </tr>
  </tbody>
</table>
</div>

# Example of the selected SMP profile snow depth compared to the snow depths of all SMP profiles and MagnaProbe distribution for the SM site (Table 2 of [Montpetit et al., 2024](Link TBD))

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
x = np.linspace(0,100,200)
site = 'SM02'

mp_site = mp.loc[(mp.site==site.replace('RP','RS'))].depth
param = gamma.fit(mp_site)
sd_mode = x[gamma.pdf(x, *param[:-2], loc = param[-2], scale = param[-1]) == gamma.pdf(x, *param[:-2], loc = param[-2], scale = param[-1]).max()]
    
best_pit = smp_profiles[(smp_profiles.site==site.replace('RP','RS'))].iloc[0].position
delta_mode = np.abs(smp_profiles[(smp_profiles.site==site.replace('RP','RS'))].iloc[0].density.rel_height.max()/10-sd_mode)

for position in smp_profiles[(smp_profiles.site==site.replace('RP','RS'))].position.values:
    
    if np.abs(smp_profiles[(smp_profiles.site==site.replace('RP','RS')) & (smp_profiles.position==position)].density.iloc[0].rel_height.max()/10-sd_mode) < delta_mode:
        
        best_pit = position
        delta_mode = np.abs(smp_profiles[(smp_profiles.site==site.replace('RP','RS')) & (smp_profiles.position==position)].density.iloc[0].rel_height.max()/10-sd_mode)

hsnow = ax.hist(mp_site, bins=25, density=True, color='k', label='MagnaProbe \nSnow Depths')

for position in smp_profiles.loc[(smp_profiles.site==site.replace('RP','RS'))].position.values:
    ax.axvline(smp_profiles.loc[(smp_profiles.site==site.replace('RP','RS')) & (smp_profiles.position==position) ].density.iloc[0].rel_height.max()/10,color='gray',label='SMP \nSnow Depths', alpha=0.7,linestyle='--',linewidth=4)
ax.axvline(smp_profiles.loc[(smp_profiles.site==site.replace('RP','RS')) & (smp_profiles.position==best_pit) ].density.iloc[0].rel_height.max()/10,color='cyan',label='Selected \nSMP Profile', alpha=0.7,linewidth=4)
ax.set_xlabel('Snow Depth (cm)', fontsize=20, fontweight='bold')
ax.set_ylabel('Probability Density Function', fontsize=20, fontweight='bold')
ax.set_ylim(0,0.07)

handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys())

<center><img src="Figures/Figure8.png" height="500px" class="bg-primary mb-1"></center>

<center>Figure 8 of [Montpetit et al. (2024)](Link TBD): Example of the SMP profile selection as a representative snowpack for the SM site (Table 2).</center>